In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Set to True to force a full refresh of the data
full_refresh = True

In [3]:
repo_root = os.popen('git rev-parse --show-toplevel').read().strip()

In [4]:
data_folder = os.path.join(repo_root, 'multi-parameter-estimation', 'data')

# Get list of data directories
data_dirs = os.listdir(data_folder)
data_dirs = [d for d in data_dirs if os.path.isdir(os.path.join(data_folder, d))]

# skip old-data
if 'old-data' in data_dirs:
    data_dirs.remove('old-data')

data_dirs.sort()
data_dirs

['2025-06-02--17h-06m-16s',
 '2025-06-02--17h-07m-37s',
 '2025-06-02--17h-07m-47s',
 '2025-06-02--17h-07m-56s',
 '2025-06-02--17h-08m-05s',
 '2025-06-02--17h-08m-19s']

In [5]:
new_data_dirs = data_dirs.copy()

if not full_refresh:
    for d in data_dirs:
        if os.path.exists(os.path.join(data_folder, d, "chunked_coincidences_n=200.csv")):
            new_data_dirs.remove(d)

new_data_dirs

['2025-06-02--17h-06m-16s',
 '2025-06-02--17h-07m-37s',
 '2025-06-02--17h-07m-47s',
 '2025-06-02--17h-07m-56s',
 '2025-06-02--17h-08m-05s',
 '2025-06-02--17h-08m-19s']

In [6]:
def load_chunks(data_dir):
    if not os.path.exists(os.path.join(data_folder, data_dir, "chunked_coincidences_n=40.csv")):
        print(f"Skipping {data_dir} as it does not have the required file.")
        return pd.DataFrame()
    coincidences = pd.read_csv(os.path.join(data_folder, data_dir, "chunked_coincidences_n=40.csv"))
    coincidences["data_dir"] = data_dir
    return coincidences

chunks_df = pd.concat([load_chunks(d) for d in new_data_dirs], ignore_index=True)
chunks_df 

,data_dir,C,DB_H,DB_V,SB,N
0,2025-06-02--17h-06m-16s,0.0,40.0,0.0,0.0,40.0
1,2025-06-02--17h-06m-16s,0.0,40.0,0.0,0.0,40.0
2,2025-06-02--17h-06m-16s,0.0,40.0,0.0,0.0,40.0
3,2025-06-02--17h-06m-16s,0.0,40.0,0.0,0.0,40.0
4,2025-06-02--17h-06m-16s,0.0,40.0,0.0,0.0,40.0
...,...,...,...,...,...,...
1749,2025-06-02--17h-08m-19s,0.0,12.0,12.0,16.0,40.0
1750,2025-06-02--17h-08m-19s,1.0,13.0,7.0,19.0,40.0
1751,2025-06-02--17h-08m-19s,0.5,12.0,10.0,17.0,39.5
1752,2025-06-02--17h-08m-19s,0.5,14.0,12.0,13.0,39.5


In [7]:
# drop columns that are not needed
def k_wise_sum(group, k):
    # Drop last rows if not divisible by k
    n = len(group) - (len(group) % k)
    group = group.iloc[:n].reset_index(drop=True)
    # Sum every k rows
    kwise = group.groupby(group.index // k).sum()
    # Restore data_dir from the first row of each group
    kwise['data_dir'] = group['data_dir'].iloc[::k].values
    return kwise

# Example usage for k=3
df_80 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=2)).reset_index(drop=True)
df_120 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=3)).reset_index(drop=True)
df_160 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=4)).reset_index(drop=True)
df_200 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=5)).reset_index(drop=True)
df_200


/tmp/ipykernel_348790/345050552.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_80 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=2)).reset_index(drop=True)
/tmp/ipykernel_348790/345050552.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_120 = chunks_df.groupby('data_dir', group_keys=False).apply(lambda g: k_wise_sum(g, k=3)).reset_index(drop=True)
/tmp/ipyk

,data_dir,C,DB_H,DB_V,SB,N
0,2025-06-02--17h-06m-16s,0.0,200.0,0.0,0.0,200.0
1,2025-06-02--17h-06m-16s,0.0,200.0,0.0,0.0,200.0
2,2025-06-02--17h-06m-16s,0.0,200.0,0.0,0.0,200.0
3,2025-06-02--17h-06m-16s,0.0,200.0,0.0,0.0,200.0
4,2025-06-02--17h-06m-16s,0.0,200.0,0.0,0.0,200.0
...,...,...,...,...,...,...
345,2025-06-02--17h-08m-19s,6.0,72.0,40.0,79.5,197.5
346,2025-06-02--17h-08m-19s,3.5,69.0,38.0,87.5,198.0
347,2025-06-02--17h-08m-19s,5.0,69.0,42.0,82.0,198.0
348,2025-06-02--17h-08m-19s,3.5,64.0,42.0,89.5,199.0


In [8]:
# save the dataframes to csv files
# save the dataframes to csv files based on the data_dir
for data_dir in new_data_dirs:
    df_80_subset = df_80[df_80["data_dir"] == data_dir]
    df_120_subset = df_120[df_120["data_dir"] == data_dir]
    df_160_subset = df_160[df_160["data_dir"] == data_dir]
    df_200_subset = df_200[df_200["data_dir"] == data_dir]
    df_80_subset.to_csv(os.path.join(data_folder, data_dir, "chunked_coincidences_n=80.csv"), index=False)
    df_120_subset.to_csv(os.path.join(data_folder, data_dir, "chunked_coincidences_n=120.csv"), index=False)
    df_160_subset.to_csv(os.path.join(data_folder, data_dir, "chunked_coincidences_n=160.csv"), index=False)
    df_200_subset.to_csv(os.path.join(data_folder, data_dir, "chunked_coincidences_n=200.csv"), index=False)